In [ ]:
import pandas
import pathlib
import rapidfuzz
import statistics

# load datasets.

df1 = pandas.read_csv(pathlib.Path.cwd() / 'data1.csv')
df2 = pandas.read_csv(pathlib.Path.cwd() / 'data2.csv')

# compile a list of unique df2 agents for name matching.

df2_agent_label_list = list(df2.agent_label.unique())

# loop through df1 agents, attempting to find matches from df2 dataset.

for df1_agent_id in df1.agent_id.unique():
    
    # build a filmography for df1 agent, respecting that multiple titles are against single works.

    df1_agent_filmography = df1.loc[df1.agent_id.isin([df1_agent_id])][['work_id']].drop_duplicates().to_dict('records')
    for film in df1_agent_filmography:
        df1_work_id = film['work_id']
        film['work_labels'] = list(df1.loc[df1.work_id.isin([df1_work_id])]['work_label'].unique())
    
    # assemble a list of candidates, which are selected based on a Levenshtein distance of 85 or greater between two names.

    agent_label_match = list()
    df1_agent_label = list(df1.loc[df1.agent_id.isin([df1_agent_id])]['agent_label'].unique())
    for label in df1_agent_label:
        string_matching = rapidfuzz.process.extract(label, df2_agent_label_list, scorer=rapidfuzz.fuzz.WRatio, processor=rapidfuzz.utils.default_process)
        agent_label_match +=  [x[0] for x in string_matching if x[1] >= 90]

    # loop through each df2 candidate, attempting to find the best title match for each filmography work.

    candidates = list(df2.loc[df2.agent_label.isin(agent_label_match)]['agent_id'].unique())
    for df2_agent_id in candidates:

        # compute all candidate titles for work title matching.

        candidate_titles = list(df2.loc[df2.agent_id.isin([df2_agent_id])]['work_label'].unique())

        candidate_score = list()
        for work in df1_agent_filmography:

            work_score = list()
            for label in work['work_labels']:
                string_matching = rapidfuzz.process.extract(label, candidate_titles, scorer=rapidfuzz.fuzz.WRatio, processor=rapidfuzz.utils.default_process)
                work_score += [x[1] for x in string_matching]

            candidate_score.append(max(work_score))

        # if candidate score matches certain characteristics, declare a match.
   
        if len(candidate_score) >= 4 and statistics.median(candidate_score) >= 90:

            df2_agent_label = list(df2.loc[df2.agent_id.isin([df2_agent_id])]['agent_label'].unique())
            print(f'{df1_agent_label[0]} ({df1_agent_id}) matched with {df2_agent_label[0]} ({df2_agent_id})')